Copyright 2023 Google LLC. SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# layout model basic inference

In [ ]:
import pickle
import torch
from models.layout import model_full
from utils import camera_util
from baukit import renormalize

torch.set_grad_enabled(False)
device = 'cuda'

full_model = model_full.ModelFull('pretrained/model_terrain.pkl', 'pretrained/model_sky_360.pkl').to(device).eval()
G_layout = full_model.terrain_model.layout_model
torch.manual_seed(1)
z = torch.randn(1, G_layout.layout_generator.z_dim, device=device)
c = None
camera_params = camera_util.get_full_image_parameters(
    G_layout, G_layout.layout_decoder_kwargs.nerf_out_res, batch_size=1, device=device)
output = full_model(z, c, camera_params)
renormalize.as_image(output['rgb_overlay_upsample'][0])

# triplane model basic inference

In [ ]:
from models.triplane import model_full
from utils.soat_util_triplane import construct_intrinsics

full_model = model_full.ModelFull('pretrained/model_triplane.pkl', 'pretrained/model_sky_360.pkl').to(device).eval()
torch.manual_seed(1)
z = torch.randn([1, full_model.ground.z_dim]).to(device)  # latent codes
intrinsics = construct_intrinsics(60).to(device)
extrinsics = full_model.world2cam_poses[1].inverse().to(device) # change world2cam -> cam2world pose
c = torch.cat([extrinsics.reshape(1, -1), intrinsics.reshape(1, -1)], dim=-1)
output = full_model(z, c, render_sky='True')
renormalize.as_image(output['composite'][0])